### This is a tutorial for running the Carlsbad case study. The steps are:
#### 1. Import WaterTAP3 package 
#### 2. Look up case study options
#### 3. Define your case study, reference, and water type based on the case study library.
#### 4. Set up model and populate model with case study information.
#### 5. View the treatment train.
#### 6. Run the treatment train.
#### 7. Look at the results table.
#### 8. View results using Dash.
#### 9. Sensitivity Analysis

##### Step 1. Import WaterTAP3 package

In [1]:
import watertap as wt

##### Step 2. Look up case study options. For now each case study has only one reference and water type and the baseline scenario is the only option. Eventually we will be able to mix and match options below. For example, selecting a train from one place and the source water from another. We may or may not have the scenario selection here. 

In [2]:
wt.case_study_trains.case_study_library.head()

,Case_Study,Scenario,Reference,WaterType
0,Ashkelon,Baseline,NAWI,Seawater
1,Carlsbad,TwoPassRO,NAWI,Seawater
2,Carlsbad,Baseline,NAWI,Seawater
3,Tampa_Bay,Baseline,NAWI,Seawater
4,Santa_Barbara,Baseline,NAWI,Seawater


##### Step 3: Define your case study, reference, and water type based on the case study library.

In [3]:
wt.case_study_trains.case_study = "Carlsbad"
wt.case_study_trains.reference = "NAWI"
wt.case_study_trains.water_type = "Seawater"
wt.case_study_trains.scenario = "TwoPassRO"

##### Step 4: Set up model and populate model with case study information. #TODO Flow is in m3/s (user can select unit of their choice - future tutorial).

In [4]:
# eventually we may want to be able to choose a dynamic model or different model set ups.
m = wt.watertap_setup(dynamic = False)

# load the treatment train, source water information, and flow to the model. #TODO automate flow based on case study.
m = wt.case_study_trains.get_case_study(flow = 4.5833, m = m)

##### Step 5: View the treatment train.

In [5]:
wt.display.show_train2(model_name=m)

##### Step 6. Run the treatment train.

In [6]:
wt.run_water_tap(m = m, solver_results = True, print_model_results = True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

#### 7. Look at the results table.

In [7]:
wt.get_results_table(m = m).tail()

,Unit Process Name,Variable,Value,Metric,Unit,Case Study,Scenario
1045,System,System Electricity,66.801,Annual Cost,$MM/yr,Carlsbad,TwoPassRO
1046,System,System Other Variable Operating,0.000,Annual Cost,$MM/yr,Carlsbad,TwoPassRO
1047,System,System Fixed Operating,6.732,Annual Cost,$MM/yr,Carlsbad,TwoPassRO
1048,System,System Total Operating Cost,83.312,Annual Cost,$MM/yr,Carlsbad,TwoPassRO
1049,System,System Levelized Cost,1.624,LCOW,$/m3,Carlsbad,TwoPassRO


#### 8. View results using Dash.

In [8]:
csv_dir = './results/CarlsbadCompare.csv'
json_dir = './tmp/example.json'

wt.app3.run_dash(csv_dir, json_dir)

Dash app running on http://127.0.0.1:8050/


##### Comparison of two scenarios

In [10]:
# See that there are two Carlsbad options, so we can compare those (others currently in progress)
wt.case_study_trains.case_study_library.head()

,Case_Study,Scenario,Reference,WaterType
0,Ashkelon,Baseline,NAWI,Seawater
1,Carlsbad,TwoPassRO,NAWI,Seawater
2,Carlsbad,Baseline,NAWI,Seawater
3,Tampa_Bay,Baseline,NAWI,Seawater
4,Santa_Barbara,Baseline,NAWI,Seawater


In [12]:
results = wt.run_model_comparison(scenarioA = "Baseline", scenarioB = "TwoPassRO", flow = 4.5833)

In [13]:
results.head()

,Unit Process Name,Variable,Value,Metric,Unit,Case Study,Scenario
0,sw_onshore_intake,Fixed Capital Investment (FCI),20.980,Cost,$MM,Carlsbad,Baseline
1,sw_onshore_intake,Land,0.031,Cost,$MM,Carlsbad,Baseline
2,sw_onshore_intake,Working Capital,1.049,Cost,$MM,Carlsbad,Baseline
3,sw_onshore_intake,Total Capital Investment (TCI),22.061,Cost,$MM,Carlsbad,Baseline
4,sw_onshore_intake,Catalysts and Chemicals,0.000,Annual Cost,$MM/yr,Carlsbad,Baseline


In [14]:
results.tail()

,Unit Process Name,Variable,Value,Metric,Unit,Case Study,Scenario
1045,System,System Electricity,66.801,Annual Cost,$MM/yr,Carlsbad,TwoPassRO
1046,System,System Other Variable Operating,0.000,Annual Cost,$MM/yr,Carlsbad,TwoPassRO
1047,System,System Fixed Operating,6.732,Annual Cost,$MM/yr,Carlsbad,TwoPassRO
1048,System,System Total Operating Cost,83.312,Annual Cost,$MM/yr,Carlsbad,TwoPassRO
1049,System,System Levelized Cost,1.624,LCOW,$/m3,Carlsbad,TwoPassRO


#### 9. Sensitivity Analysis

In [ ]:
import numpy as np
import pandas as pd
from pyomo.environ import value
import time

In [ ]:
#tds_range = np.arange(1, 200)
#backwash_scenario = [1e-5, 0.5, 0.95]

tds_range = np.arange(25, 85)
backwash_scenario = [0.95]
pmax_range = np.arange(75, 95)

In [ ]:
startTime = time.time()

lcow_list = []
recovery_list = []
scenario_list = []
tds_list = []

wt.case_study_trains.case_study = "Carlsbad"
wt.case_study_trains.reference = "NAWI"
wt.case_study_trains.water_type = "Seawater"
wt.case_study_trains.scenario = "TwoPassRO"

final_df = pd.DataFrame()

for pmax_scenario in pmax_range:

    for bw_scenario in backwash_scenario:

        #for value_change in pct_to_target1: # cycles through each value from MC range
        for tds_value in tds_range:

            # create and build model
            m = wt.watertap_setup(dynamic = False)
            m = wt.case_study_trains.get_case_study(flow = 4.5833, m = m)

            m.fs.backwash_solids_handling.water_recovery.fix(bw_scenario)
            m.fs.ro_deep.pmax.fix(pmax_scenario)
            m.fs.source1.conc_mass_in[0, "TDS"].fix(tds_value)

            wt.run_water_tap(m = m, solver_results = False, print_model_results = False)

            #df = wt.get_results_table(m = m)
            #df[df["Unit Process Name"] == "System"]
            
            if "j" in str(value(m.fs.costing.LCOW)):
                continue
            else:
                inlet_flow = value(m.fs.municipal_drinking.flow_vol_in[0])
                outlet_flow = value(m.fs.source1.flow_vol_in[0])
                recovery_list.append(inlet_flow / outlet_flow)
                scenario_list.append(pmax_scenario)
                tds_list.append(value(m.fs.source1.conc_mass_in[0, "TDS"]))
                lcow_list.append(value(m.fs.costing.LCOW))

final_df["sys_recovery"] = recovery_list
final_df["lcow"] = lcow_list
final_df["tds_inlet"] = tds_list
final_df["p_max"] = scenario_list

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
final_df.to_csv("sens_test.csv")

In [ ]:
df.to_csv("")

In [ ]:
#### DO NOT USE THE BELOW ####

In [ ]:
cost_range_list = []; #results will be inputted in this array
#up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.

#for value_change in pct_to_target1: # cycles through each value from MC range
for value_change in [0.4, 0.8]: #, 0.9]:

    # create and build model
    m = wt.watertap_setup(dynamic = False)
    m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

    m.fs.tri_media_filtration.water_recovery.fix(value_change)

    # set variable to MC value
    wt.run_water_tap(m)
    results_table = get_results_table(m, unit_process_name)
    cost_range_list.append(results_table.total_up_cost.sum())


In [ ]:
import time
from multiprocessing import Pool
import multiprocessing

mu = 0.6
sigma = .1
num_reps = 50

input_list = np.random.normal(mu,sigma, size = num_reps) #, sigma, num_reps).round(4)

count, bins, ignored = plt.hist(input_list, 25, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
          linewidth=2, color='r')
plt.show()

### INPUT TO MODEL LIST: ### CAN BE AUTOMATED FOR USER TO LABEL THE VARIABLE. TOOD ###
no_of_proc = 4
list_final = []
for i in range(no_of_proc):
    part2 = len(input_list) / no_of_proc
    i2 = ((i+1)*part2)
    list1 = input_list[int(i*part2):int(i2)]
    list_final.append(list1)
    
    
def monte_run(list_final):
    print('goes in')

    up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.
    cost_range_list = []; #results will be inputted in this array

    #for value_change in pct_to_target1: # cycles through each value from MC range
    for value_change in list_final:

        # create and build model
        m = wt.watertap_setup(dynamic = False)
        m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

        getattr(m.fs, up_name).water_recovery.fix(value_change)

        # set variable to MC value
        result = wt.run_water_tap(m)
        results_table = get_results_table(m, unit_process_names)
        cost_range_list.append(results_table.total_up_cost.sum())


    return cost_range_list

startTime = time.time()

pool=Pool()
dfs = pool.map(monte_run, list_final) #SomeClass().preprocess_data()

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
2.37 / 2.39

In [ ]:
####TO DO LOAD AND SAVE!!

In [ ]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [ ]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [ ]:
# wt.display.show_train(TT)